# <div align="center" style="color: #ff5733;">Purple Key Query</div>

In [3]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

C:\Users\DwaipayanChakroborti\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
sq = """
WITH 
W_VAS_FEES
AS
(
  select loanId, sum(scheduleChargeAmount) as VasFees  from prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_repayment_schedule  f
  group by 1
)
,
pos 
AS 
(select 
PurpleKey, 
BrandStores, 
POS, 
Region, 
DateOpened datestoreopened, 
TSM OpeningTSM
from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING),
promoterloan
AS    
(select * from 
  (select promoterId, digitalLoanAccountId,createdDate,
row_number() over(partition by promoterId, digitalLoanAccountId order by createdDate desc) rnk
  from prj-prod-dataplatform.dl_loans_db_raw.tdbk_qr_promoter_details where promoterId is not null
  ) where rnk = 1
),
promotermaster 
AS 
(select distinct Promoter_Employee_Number, Promoter_Name, Promoter_Type, Promoter_Status , Access_Code, Promote_Date_of_Joining , End_of_Employment
from prj-prod-dataplatform.dap_ds_poweruser_playground.Promoter_master)
,
promoter 
AS    
(select a.promoterId, 
pm.Promoter_Employee_Number, 
a.digitalLoanAccountId, 
pm.Promoter_Name , 
pm.Promoter_Type, 
pm.Promoter_Status , 
pm.Access_Code, 
pm.Promote_Date_of_Joining , 
pm.End_of_Employment,
a.createdDate,
row_number() over(partition by digitalLoanAccountId order by createdDate desc) rnk
from promoterloan a
left join promotermaster pm on a.promoterId = pm.Access_Code
where digitalLoanAccountId is not null and Promoter_Employee_Number is not null
),
loans 
AS    
( SELECT a.digitalLoanAccountId
, a.startApplyDateTime
, a.purposeDescription
, CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN a.cddVerificationDateAndTime
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL THEN COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime)
    ELSE NULL END AS decision_date
, a.disbursementDateTime
, a.purpleKey
, d. BrandStores
, d.Region 
, a.merchantName
, a.stage1status
, a.approvedLoanAmount
, a.downPaymentAmount
,(coalesce(cast(c.price as float64),0)) as Items_Sold_Price
, a.isTermsAndConditionsAccepted
, CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN a.flagApproval
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL AND a.stage1status in ('EXPIRED','APPROVED') THEN 1 
    ELSE 0 END AS flagApproval
, CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN flagRejection
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL AND COALESCE(a.stage1status,'XNA') NOT in ('EXPIRED','APPROVED') THEN 1 
    ELSE 0 END AS flagRejection
, a.flagDisbursement
, a.disbursedLoanTenur
, a.disbursedLoanAmount
, a.BNPL_PRODUCT_CATEGORY_CNT
, a.employmentstatus
, a.new_loan_type
, B.Promoter_Employee_Number
, coalesce(initcap(b.Promoter_Name) ,'XNA') as Promoter
, b.End_of_Employment Promoter_LWD 
, d.OpeningTSM
, d.datestoreopened
, case when e.vas_flag = 'true' then 1 else 0 end as VasFlag
, f.VasFees
, a.defFPD30
, a.obsFPD30
, a.defSPD30
, a.obsSPD30
, a.defTPD30
, a.obsTPD30
    FROM 
    prj-prod-dataplatform.risk_credit_mis.loan_master_table a
    left join promoter b
    on a.digitalLoanAccountId = b.digitalLoanAccountId and b.rnk = 1
    left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category c 
    on a.digitalLoanAccountId = c.digitalLoanAccountId
    left join pos d
    on d.purpleKey = a.purpleKey
    left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
    on e.loanAccountNumber = a.loanAccountNumber
    left join W_VAS_FEES  f
    on a.loanAccountNumber = f.loanId# and f.repaymentNumber = 1
    WHERE
    a.new_loan_type = 'SIL-Instore'
    AND
    DATE_TRUNC(a.startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)),
W_LOAN_MASTER
AS
(
  SELECT digitalLoanAccountId
  , startApplyDateTime
  , DECISION_DATE
  , disbursementDateTime
  , purpleKey
  , BrandStores
  , merchantName
  , purposeDescription as Commodity
  , case when stage1status in ('EXPIRED','APPROVED') then approvedLoanAmount end as approvedLoanAmount
  , case when stage1status in ('EXPIRED','APPROVED') then CAST(downPaymentAmount AS FLOAT64) end as downPaymentAmount
  , COALESCE(isTermsAndConditionsAccepted,0) AS APP_COMPLETED
  , flagApproval
  , CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 AND stage1status in ('APPROVED') THEN 1 ELSE 0 END AS APPLICATION_APPROVED_NOT_BOOKED_CNT
  , CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 AND stage1status in ('EXPIRED') THEN 1 ELSE 0 END AS APPLICATION_EXPIRED
  , flagRejection
  , flagDisbursement
  , disbursedLoanTenur
  , disbursedLoanAmount, 
  CASE WHEN COALESCE(BNPL_PRODUCT_CATEGORY_CNT,0) > 1 THEN 'Y' Else 'N' END AS BUNDLED
  ,  Initcap(employmentstatus) AS employmentstatus
  , Promoter_Employee_Number
  , Promoter
  , Promoter_LWD
  , OpeningTSM
  , datestoreopened,
  sum(case when stage1status in ('EXPIRED','APPROVED') then Items_Sold_Price end) as Items_Sold_Price,
  max(VasFlag) as VasFlag ,
  max(VasFees) as VasFees 
  from loans
   WHERE
  new_loan_type = 'SIL-Instore'
  AND
  DATE_TRUNC(startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)
  Group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24, 25
)
,
W_DTS
AS
(
    SELECT DISTINCT DATE_TRUNC(startApplyDateTime,DAY) AS DT, DATE_TRUNC(startApplyDateTime,ISOWEEK) AS WEEK, DATE_TRUNC(startApplyDateTime,MONTH) AS MONTH
    FROM prj-prod-dataplatform.risk_credit_mis.loan_master_table 
    WHERE
    DATE_TRUNC(startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)
)
,
delq_base as
(select lmt.loanAccountNumber, lmt.digitalLoanAccountId,
sum(case when ldd.min_inst_def30 = 1 then 1 else 0 end) fpd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 1 then 1 else 0 end) obs_fpd30_cnt,
sum(case when ldd.min_inst_def30 = 1 then lmt.disbursedloanamount else 0 end) fpd30_v,
sum(case when ldd.obs_min_inst_def30 >= 1 then lmt.disbursedloanamount else 0 end) obs_FPD30_v,
sum(case when ldd.min_inst_def30 = 2 then 1 else 0 end) spd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 2 then 1 else 0 end) obs_spd30_cnt,
sum(case when ldd.min_inst_def30 = 2 then lmt.Total_Outstanding_Principal else 0 end) spd30_v,
sum(case when ldd.obs_min_inst_def30 >= 2 then lmt.disbursedloanamount else 0 end) obs_spd30_v,
sum(case when ldd.min_inst_def30 = 3 then 1 else 0 end) tpd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 3 then 1 else 0 end) obs_tpd30_cnt,
sum(case when ldd.min_inst_def30 = 3 then lmt.Total_Outstanding_Principal else 0 end) tpd30_v,
sum(case when ldd.obs_min_inst_def30 >=3 then lmt.disbursedloanamount else 0 end) obs_tpd30_v,
sum(case when ldd.obs_min_inst_def30 >= 3 and ldd.min_inst_def30 in (1,2,3) then 1 else 0 end) fstpd30_cnt,
sum(Case when ldd.obs_min_inst_def30 >= 3 then 1 else 0 end) obs_fstpd30_cnt
from
`risk_credit_mis.loan_master_table` lmt
left join `prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data` ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
where lmt.flagDisbursement = 1
group by 1,2
),
loanbase as 
(select a.digitalLoanAccountId, a.startApplyDateTime, 
a.DECISION_DATE,
a.disbursementDateTime,
a.purpleKey,
a.merchantName,
a.BrandStores,
a.approvedLoanAmount,
a.downPaymentAmount,
a.APP_COMPLETED,
a.flagApproval,
a.APPLICATION_APPROVED_NOT_BOOKED_CNT,
a.APPLICATION_EXPIRED,
a.flagRejection,
a.flagDisbursement,
a.disbursedLoanTenur,
a.disbursedLoanAmount,
a.BUNDLED,
a.employmentstatus,
a.Promoter_Employee_Number,
a.Promoter,
a.Promoter_LWD,
a.Commodity,
a.OpeningTSM,
a.datestoreopened,
a.Items_Sold_Price,
a.VasFlag,
a.VasFees,
db.loanAccountNumber,
db.fpd30_cnt,
db.obs_fpd30_cnt,
db.fpd30_v,
db.obs_FPD30_v,
db.spd30_cnt,
db.obs_spd30_cnt,
db.spd30_v,
db.obs_spd30_v,
db.tpd30_cnt,
db.obs_tpd30_cnt,
db.tpd30_v,
db.obs_tpd30_v,
db.fstpd30_cnt,
db.obs_fstpd30_cnt
from W_LOAN_MASTER a
left join delq_base db on db.digitalLoanAccountId = a.digitalLoanAccountId
),
W_MERCHANTS
AS
(
    SELECT DISTINCT purpleKey AS MERCHANT_CODE
    , merchantName as Store
    , BrandStores as Merchant
    , CASE WHEN purplekey IN ('M9ZK6IU','MV4UO0X','M2G78K4','MEKA38B','MOE3JQH') THEN 'N' ELSE 'Y' END AS ACTIVE
    , COALESCE(OpeningTSM,'XNA') AS TSM
    , datestoreopened DateOpened
    , BUNDLED
    , employmentstatus
    , Promoter_Employee_Number
    , Promoter
    , Promoter_LWD
    , Commodity
    FROM loanbase
) 
,
W_DTS_MERCHANT 
AS
(
    SELECT W_DTS.DT, W_DTS.WEEK, W_DTS.MONTH, W_MERCHANTS.MERCHANT_CODE, W_MERCHANTS.Store, W_MERCHANTS.Merchant, W_MERCHANTS.ACTIVE,
    CASE WHEN W_DTS.DT BETWEEN CURRENT_DATE - 7 AND CURRENT_DATE - 1 THEN 'Y' ELSE 'N' END AS LAST_7_DAYS,
    CASE WHEN W_DTS.WEEK BETWEEN DATE_TRUNC(CURRENT_DATE - 49 - 1, ISOWEEK) AND CURRENT_DATE - 1 THEN 'Y' ELSE 'N' END AS LAST_8_WEEKS,
    CASE WHEN W_DTS.MONTH BETWEEN DATE_SUB( DATE_TRUNC(CURRENT_DATE - 1, MONTH), INTERVAL 3 MONTH) AND CURRENT_DATE - 1 THEN 'Y' ELSE 'N' END AS LAST_4_MONTHS    ,
    BUNDLED, TSM, W_MERCHANTS.employmentstatus, W_MERCHANTS.Promoter_Employee_Number, W_MERCHANTS.Promoter, W_MERCHANTS.Promoter_LWD, W_MERCHANTS.DateOpened, W_MERCHANTS.Commodity
    FROM
    W_DTS CROSS JOIN W_MERCHANTS
)
,
W_STARTED
AS
(
    SELECT DATE_TRUNC(startApplyDateTime,DAY) AS APP_START_DATE, 
    purpleKey, 
    BUNDLED, 
    employmentstatus, 
    Promoter_Employee_Number, 
    Promoter, 
    Promoter_LWD, 
    Commodity,
    COUNT(distinct digitalLoanAccountId) AS APP_STARTED, 
    SUM(APP_COMPLETED) AS APP_COMPLETED, 
    SUM(APPLICATION_APPROVED_NOT_BOOKED_CNT) AS APPLICATION_APPROVED_NOT_BOOKED_CNT,
    SUM(APPLICATION_EXPIRED) AS APPLICATION_EXPIRED,
    SUM(CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 THEN approvedLoanAmount ELSE 0 END) AS APPROVED_NOT_BOOKED_VOLUME
    FROM loanbase
    WHERE startApplyDateTime IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8
)
,
W_DECIDED
AS
(
    SELECT DATE_TRUNC(DECISION_DATE,DAY) AS APP_DECISION_DATE, purpleKey,BUNDLED, employmentstatus, Promoter_Employee_Number, Promoter, Promoter_LWD, Commodity,
    COUNT(distinct digitalLoanAccountId) AS APP_DECIDED, SUM(flagApproval) AS APP_APPROVED, SUM(flagRejection) AS APP_REJECTED, 
    SUM(approvedLoanAmount) AS approvedLoanAmount, SUM(downPaymentAmount) AS downPaymentAmount, SUM(Items_Sold_Price) AS Items_Sold_Price
    FROM loanbase
    WHERE DECISION_DATE IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8
)
,
W_BOOKED
AS
(
    SELECT DATE_TRUNC(disbursementDateTime,DAY) AS APP_BOOKED_DATE, purpleKey, BUNDLED, employmentstatus, Promoter_Employee_Number, Promoter, Promoter_LWD, Commodity,
    SUM(flagDisbursement) AS APP_BOOKED, SUM(disbursedLoanAmount) AS APP_BOOKED_VOLUME, 
    SUM(disbursedLoanTenur) AS APP_BOOKED_TENURE, 
    SUM(CASE WHEN disbursedLoanTenur > 3 THEN 1 ELSE 0 END) AS VAS_ELIGIBLE_BASE,
    SUM(VasFlag) AS VAS_SOLD, SUM(VasFees) AS VAS_SOLD_VOLUME,
    SUM(fpd30_cnt) fpd30_cnt,
    SUM(obs_fpd30_cnt) obs_fpd30_cnt,
    SUM(fpd30_v) fpd30_v,
    SUM(obs_FPD30_v) obs_FPD30_v,
    SUM(spd30_cnt) spd30_cnt,
    sum(obs_spd30_cnt) obs_spd30_cnt,
    SUM(spd30_v) spd30_v,
    SUM(obs_spd30_v) obs_spd30_v,
    SUM(tpd30_cnt) tpd30_cnt,
    SUM(obs_tpd30_cnt)obs_tpd30_cnt,
    SUM(tpd30_v) tpd30_v,
    SUM(obs_tpd30_v) obs_tpd30_v,
    SUM(fstpd30_cnt) fstpd30_cnt,
    SUM(obs_fstpd30_cnt) obs_fstpd30_cnt
    FROM loanbase
    WHERE disbursementDateTime IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8
)
,
Querybase as 
(SELECT
EXTRACT(DATE FROM W_DTS_MERCHANT.DT) AS DT, 
EXTRACT(DATE FROM W_DTS_MERCHANT.WEEK) AS WEEK, 
EXTRACT(DATE FROM W_DTS_MERCHANT.MONTH) AS MONTH, 
W_DTS_MERCHANT.LAST_7_DAYS AS LAST_7_DAYS,
W_DTS_MERCHANT.LAST_8_WEEKS AS LAST_8_WEEKS,
W_DTS_MERCHANT.LAST_4_MONTHS AS LAST_4_MONTHS,
W_DTS_MERCHANT.Store as Store, 
W_DTS_MERCHANT.Merchant AS Merchant,
W_DTS_MERCHANT.ACTIVE AS ActiveStore,
W_DTS_MERCHANT.BUNDLED AS BUNDLED,
W_DTS_MERCHANT.TSM AS TSM,
W_DTS_MERCHANT.employmentstatus AS employmentstatus,
W_DTS_MERCHANT.Promoter AS Promoter,
W_DTS_MERCHANT.Promoter_LWD AS Promoter_LWD,
W_DTS_MERCHANT.DateOpened AS DateOpened,
W_DTS_MERCHANT.Commodity AS Commodity,
COALESCE(SUM(W_STARTED.APP_STARTED),0) AS APPLICATION_CNT,
COALESCE(SUM(W_STARTED.APP_COMPLETED),0) AS APP_COMPLETED_CNT,
COALESCE(SUM(W_STARTED.APPLICATION_APPROVED_NOT_BOOKED_CNT),0) AS APPLICATION_APPROVED_NOT_BOOKED_CNT, 
COALESCE(SUM(W_STARTED.APPLICATION_EXPIRED),0) AS APPLICATION_EXPIRED, 
COALESCE(SUM(W_STARTED.APPROVED_NOT_BOOKED_VOLUME),0) AS APPROVED_NOT_BOOKED_VOLUME,
COALESCE(SUM(W_DECIDED.APP_DECIDED),0) AS APPLICATION_DECISION_CNT, 
COALESCE(SUM(W_DECIDED.APP_APPROVED),0) AS APPLICATION_APPROVED_CNT, 
COALESCE(SUM(W_DECIDED.APP_REJECTED),0) AS APPLICATION_REJECTED_CNT, 
COALESCE(SUM(W_DECIDED.approvedLoanAmount),0) AS approvedLoanAmount, 
COALESCE(SUM(W_DECIDED.downPaymentAmount),0) AS downPaymentAmount,
COALESCE(SUM(W_DECIDED.Items_Sold_Price),0) AS Items_Sold_Price,
COALESCE(SUM(W_BOOKED.APP_BOOKED),0) AS APPLICATION_BOOKING_CNT,
COALESCE(SUM(W_BOOKED.APP_BOOKED_VOLUME),0) AS BOOKED_VOLUME,
COALESCE(SUM(W_BOOKED.APP_BOOKED_TENURE),0) AS APPLICATION_BOOKED_TENURE,
COALESCE(SUM(W_BOOKED.VAS_ELIGIBLE_BASE),0) AS VAS_ELIGIBLE_BASE,
COALESCE(SUM(W_BOOKED.VAS_SOLD),0) AS VAS_SOLD,
COALESCE(SUM(W_BOOKED.VAS_SOLD_VOLUME),0) AS VAS_SOLD_VOLUME,
coalesce(SUM(W_BOOKED.fstpd30_cnt), 0) AS fstpd30_cnt,
COALESCE(SUM(W_BOOKED.obs_fstpd30_cnt), 0) AS obs_fstpd30_cnt
FROM
W_DTS_MERCHANT
LEFT JOIN W_STARTED ON W_STARTED.APP_START_DATE = W_DTS_MERCHANT.DT AND COALESCE(W_STARTED.purpleKey,'XNA') like COALESCE(W_DTS_MERCHANT.MERCHANT_CODE,'XNA') 
AND COALESCE(W_STARTED.BUNDLED,'XNA') = COALESCE(W_DTS_MERCHANT.BUNDLED,'XNA') AND COALESCE(W_STARTED.employmentstatus,'XNA') = COALESCE(W_DTS_MERCHANT.employmentstatus,'XNA') AND COALESCE(W_STARTED.Promoter_Employee_Number,'XNA') = COALESCE(W_DTS_MERCHANT.Promoter_Employee_Number,'XNA') AND COALESCE(W_STARTED.Commodity,'XNA') = COALESCE(W_DTS_MERCHANT.Commodity,'XNA')
LEFT JOIN W_DECIDED ON W_DECIDED.APP_DECISION_DATE = W_DTS_MERCHANT.DT AND COALESCE(W_DECIDED.purpleKey,'XNA') = COALESCE(W_DTS_MERCHANT.MERCHANT_CODE,'XNA') 
AND COALESCE(W_DECIDED.BUNDLED,'XNA') = COALESCE(W_DTS_MERCHANT.BUNDLED,'XNA') AND COALESCE(W_DECIDED.employmentstatus,'XNA') = COALESCE(W_DTS_MERCHANT.employmentstatus,'XNA') AND COALESCE(W_DECIDED.Promoter_Employee_Number,'XNA') = COALESCE(W_DTS_MERCHANT.Promoter_Employee_Number,'XNA') AND COALESCE(W_DECIDED.Commodity,'XNA') = COALESCE(W_DTS_MERCHANT.Commodity,'XNA')
LEFT JOIN W_BOOKED ON W_BOOKED.APP_BOOKED_DATE = W_DTS_MERCHANT.DT AND COALESCE(W_BOOKED.purpleKey,'XNA') = COALESCE(W_DTS_MERCHANT.MERCHANT_CODE,'XNA') 
AND COALESCE(W_BOOKED.BUNDLED,'XNA') = COALESCE(W_DTS_MERCHANT.BUNDLED,'XNA') AND COALESCE(W_BOOKED.employmentstatus,'XNA') = COALESCE(W_DTS_MERCHANT.employmentstatus,'XNA') AND COALESCE(W_BOOKED.Promoter_Employee_Number,'XNA') = COALESCE(W_DTS_MERCHANT.Promoter_Employee_Number,'XNA') AND COALESCE(W_BOOKED.Commodity,'XNA') = COALESCE(W_DTS_MERCHANT.Commodity,'XNA')
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
)
(select 
1 sn,
'Application' type 
, qb.Month
, sst.Dates
, round(max(sst.Applications),0) Target
, sum(APPLICATION_CNT) value
, round(sum(APPLICATION_CNT) / round(max(sst.Applications),0), 2) Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(round(max(sst.Applications),0) - sum(APPLICATION_CNT), 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', date(sst.Dates)) = format_date('%Y-%m', date(qb.Month))
where sst.Dates is not null
group by 1,2,3,4
)
union all 
(select 
2 sn,
'Booked' type
, qb.Month
, sst.Dates
, round(max(sst.Booked),0) Target
, sum(qb.APPLICATION_BOOKING_CNT) value
, round(sum(qb.APPLICATION_BOOKING_CNT) / round(max(sst.Booked),0), 2) Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(round(max(sst.Booked),0) - sum(APPLICATION_BOOKING_CNT), 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
3 sn,
'Volume' type
, qb.Month
, sst.Dates
, round(max(sst.Volume), 0) Target
, sum(qb.BOOKED_VOLUME)  value
, round(sum(qb.BOOKED_VOLUME) / round(max(sst.Volume),0), 2) Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(round(max(sst.Volume),0) - sum(BOOKED_VOLUME), 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
4 sn,
'Average Finance Amount' type
, qb.Month
, sst.Dates
, round(max(sst.AverageFinanceAmount), 0) Target
, case when sum(APPLICATION_BOOKING_CNT) = 0 then 0 else round(sum(qb.BOOKED_VOLUME)/sum(APPLICATION_BOOKING_CNT),0) end  value
, case when round(max(sst.AverageFinanceAmount),0) = 0 then 0 else round(round(sum(qb.BOOKED_VOLUME)/sum(APPLICATION_BOOKING_CNT),0)/round(max(sst.AverageFinanceAmount),0), 2) end Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(round(max(sst.AverageFinanceAmount),0) -  round(sum(qb.BOOKED_VOLUME)/sum(APPLICATION_BOOKING_CNT),0), 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
5 sn,
'Downpayment' type
, qb.Month
, sst.Dates
, round(max(sst.Downpayment), 0) Target
, sum(downPaymentAmount)   value
, case when max(sst.Downpayment) = 0 then 0 else round(sum(downPaymentAmount)/max(sst.Downpayment), 2) end Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(max(sst.Downpayment) -  sum(downPaymentAmount), 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
6 sn,
'Risk (FSTPD30)' type
, qb.Month
, sst.Dates
, round(max(sst.FSTPD30), 0) Target
, CASE WHEN SUM(obs_fstpd30_cnt) = 0 THEN 0 ELSE ROUND(sum(fstpd30_cnt)/SUM(obs_fstpd30_cnt), 2) END value
, case when max(sst.FSTPD30) = 0 then 0 else ROUND(sum(fstpd30_cnt)/SUM(obs_fstpd30_cnt)/max(sst.FSTPD30), 2) end Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(max(sst.FSTPD30) -  CASE WHEN SUM(obs_fstpd30_cnt) = 0 THEN 0 ELSE ROUND(sum(fstpd30_cnt)/SUM(obs_fstpd30_cnt), 2) END, 0) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
7 sn,
'Approval Rate' type
, qb.Month
, sst.Dates
, max(sst.ApprovalRate) Target
, CASE WHEN SUM(APPLICATION_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_APPROVED_CNT)/SUM(APPLICATION_CNT), 2) END value
, case when max(sst.ApprovalRate) = 0 then 0 else ROUND(CASE WHEN SUM(APPLICATION_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_APPROVED_CNT)/SUM(APPLICATION_CNT), 2) END / max(sst.ApprovalRate), 2) end Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(max(sst.ApprovalRate) - CASE WHEN SUM(APPLICATION_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_APPROVED_CNT)/SUM(APPLICATION_CNT), 2) END , 3) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
union all
(select 
8 sn,
'Sign Rate' type
, qb.Month
, sst.Dates
, max(sst.SignRate) Target
, CASE WHEN SUM(APPLICATION_APPROVED_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_BOOKING_CNT)/SUM(APPLICATION_APPROVED_CNT), 2) END value
, case when max(sst.SignRate) = 0 then 0 else CASE WHEN SUM(APPLICATION_APPROVED_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_BOOKING_CNT)/SUM(APPLICATION_APPROVED_CNT), 2) END /max(sst.SignRate) end Actualvstarget
, max(EXTRACT(DAY FROM CURRENT_DATE()) - 1) AS days_since_beginning_of_month
, max(sst.Noofdaysinmonth) Noofdaysinmonth
, round(max(sst.SignRate) - CASE WHEN SUM(APPLICATION_APPROVED_CNT) = 0 THEN 0 ELSE ROUND(SUM(APPLICATION_BOOKING_CNT)/SUM(APPLICATION_APPROVED_CNT), 2) END , 3) Variance
from querybase qb 
left join prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_sales_target sst on format_date('%Y-%m', sst.Dates) = format_date('%Y-%m', qb.Month)
where sst.Dates is not null
group by 1,2,3,4
)
order by 1
"""

In [13]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 392bf35d-3a84-414f-b5c1-adf92a28e8f4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [14]:
pd.set_option("display.max_rows", None)

In [15]:
df

,sn,type,Month,Dates,Target,value,Actualvstarget,days_since_beginning_of_month,Noofdaysinmonth,Variance
0,1,Application,2024-02-01,2024-02-01,6.845000e+03,7185.00,1.05,23,29,-3.400000e+02
1,1,Application,2024-04-01,2024-04-01,2.247900e+04,11110.00,0.49,23,30,1.136900e+04
2,1,Application,2024-01-01,2024-01-01,8.464000e+03,8343.00,0.99,23,31,1.210000e+02
3,1,Application,2024-03-01,2024-03-01,1.442200e+04,9244.00,0.64,23,31,5.178000e+03
4,2,Booked,2024-01-01,2024-01-01,4.655000e+03,4655.00,1.00,23,31,0.000000e+00
5,2,Booked,2024-03-01,2024-03-01,7.931000e+03,5569.00,0.70,23,31,2.362000e+03
6,2,Booked,2024-04-01,2024-04-01,1.203000e+04,6340.00,0.53,23,30,5.690000e+03
7,2,Booked,2024-02-01,2024-02-01,3.765000e+03,3765.00,1.00,23,29,0.000000e+00
8,3,Volume,2024-02-01,2024-02-01,3.995418e+07,39953879.00,1.00,23,29,3.010000e+02
9,3,Volume,2024-04-01,2024-04-01,1.318200e+08,62353544.00,0.47,23,30,6.946646e+07
